In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from detector.anchor_generator import AnchorGenerator

# Generate anchors

In [ ]:
tf.reset_default_graph()

anchor_generator = AnchorGenerator(
    strides=[8, 16, 32, 64, 128],
    scales=[32, 64, 128, 256, 512],
    scale_multipliers=[1.0, 1.4142],
    aspect_ratios=[1.0, 2.0, 0.5]
)
num_anchors_per_location = anchor_generator.num_anchors_per_location

HEIGHT, WIDTH = 640, 640
postprocessing_anchors = anchor_generator(HEIGHT, WIDTH)
raw_anchors = anchor_generator.raw_anchors

In [ ]:
with tf.Session() as sess:
    anchors = sess.run(raw_anchors)
    
level = 7
stride = 2**level

anchors = anchors[level - 3]
h = int(np.ceil(HEIGHT/stride))
w = int(np.ceil(WIDTH/stride))

anchors = anchors.reshape((h, w, num_anchors_per_location, 4))
anchors.shape

# Show non clipped anchors

In [ ]:
ymin, xmin, ymax, xmax = [anchors[:, :, :, i] for i in range(4)]

h, w = (ymax - ymin), (xmax - xmin)
cy, cx = ymin + 0.5*h, xmin + 0.5*w

centers = np.stack([cy, cx], axis=3)
anchor_sizes = np.stack([h, w], axis=3)

In [ ]:
fig, ax = plt.subplots(1, dpi=100, figsize=(int(5*WIDTH/HEIGHT), 5))
unique_centers = centers[:, :, 0, :].reshape(-1, 2)
unique_sizes = anchor_sizes[0, 0, :, :]

i = 5
for j, point in enumerate(unique_centers):
    cy, cx = point
    color = 'g' if j == i else 'r' 
    ax.plot([cx], [cy], marker='o', markersize=3, color=color)

cy, cx = unique_centers[i] 
for box in unique_sizes:
    h, w = box
    xmin, ymin = cx - 0.5*w, cy - 0.5*h
    rect = plt.Rectangle(
        (xmin, ymin), w, h,
        linewidth=1.0, edgecolor='k', facecolor='none'
    )
    ax.add_patch(rect)

plt.xlim([0, WIDTH]);
plt.ylim([0, HEIGHT]);